libreries needed for the esecution of the program

    numpy

    nltk

    beautifulsoup

    sklearn

    lxml

In [1]:
#imports needed for the management of natuaral lexicon
import nltk
#nltk.download('all')
import numpy
import random
import string
from nltk import pos_tag
from nltk.corpus import stopwords

#imports needed for the management of web pages
import bs4 as bs
import urllib.request
import re

#imports needed for the management of matrixes
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#acquisition of text through webpages
html_lines1=urllib.request.urlopen('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/myth-busters')
html_lines2=urllib.request.urlopen('https://www.bbc.co.uk/bitesize/articles/zb23382')
html_lines3=urllib.request.urlopen('https://en.wikipedia.org/wiki/Coronavirus_disease_2019')
html_lines4=urllib.request.urlopen('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public')

html_content1=bs.BeautifulSoup(html_lines1.read(), 'lxml')
html_content2=bs.BeautifulSoup(html_lines2.read(), 'lxml')
html_content3=bs.BeautifulSoup(html_lines3.read(), 'lxml')
html_content4=bs.BeautifulSoup(html_lines4.read(), 'lxml')



#identification of content and creation of usable text
fake=html_content1.find_all('p') + html_content2.find_all('p')
info=html_content3.find_all('p') + html_content4.find_all('p')

infotext=''
for section in info:
    infotext = infotext + section.text

fakenews=''
for paragraph in fake:
    fakenews = fakenews + paragraph.text

In [3]:
#normalizzation of the texts
fake_sentence=nltk.sent_tokenize(fakenews)
info_sentence=nltk.sent_tokenize(infotext)

#lemmatization and remotion of stop words
lemmatizer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens if token not in set(stopwords.words('english'))]

remove_punct_dict=dict((ord(punct), None) for punct in string.punctuation)

#normalization function
def LemNormalize(a):
    return LemTokens(nltk.word_tokenize(a.lower().translate(remove_punct_dict)))

In [4]:
#transformation of texts into matrixes

def processing(sentence):
    
    #conversion into matrix
    cv=CountVectorizer(max_features=100, tokenizer=LemNormalize, analyzer='word') #normalization and vectorization

    X=cv.fit_transform(sentence) #turning text into a matrix; rows=sentences; columns=lems
    
    #array which components are the similarity coefficients between input ([X-1]), and other rows of the matrix (KB)
    vals_cv=cosine_similarity(X[-1],X) 
                
    return vals_cv    

In [5]:
#functions used to add the user query to the KB
def add_info(string):
    info_sentence.append(string)
    
def add_hoax(string):
    fake_sentence.append(string)
    
    
#selectrion of KB in response to query & creation of similarity indexes arays
def response(user_response):
    
    #adding the user's input to the text matrixes
    add_hoax(user_response)
    add_info(user_response)
        
    #searching of the highest similarity value into the fake_sentences KB
    fake_cv=processing(fake_sentence)
    
    #rearrange of the sentences by the higher similarity coefficient
    fake_more_similar=fake_cv.argsort()[0][-2]
    
    #adapting the consine simlarity array into rows of flat_vals_cv array
    fake_flat_vals_cv=fake_cv.flatten()
    
    #rearrange values by the higher cosine similarity
    fake_flat_vals_cv.sort()
    
    #return the second highest consine similarity
    hoax_similarity_index=fake_flat_vals_cv[-2]   
    
    #RIELABORO L'INDICE SULL'ALTRA KB
    info_cv=processing(info_sentence)
    info_more_similar=info_cv.argsort()[0][-2]
    info_flat_vals_cv=info_cv.flatten()
    info_flat_vals_cv.sort()
    info_similarity_index=info_flat_vals_cv[-2]
    
    
    if(hoax_similarity_index==0 and info_similarity_index==0):
        #if both searches return 0 it means they failed and there's no pertinent information in both KB
        chat_bot_response="I have no information regarding this topic \n"
        return chat_bot_response
    else:
        if(hoax_similarity_index > info_similarity_index):
            #if the search succeded returns the sentence with the highest similarity value into the fake_sentences KB
            print("the information you required has been confirmed as an hoax by trusted sources: \n")
        
            chat_bot_response=fake_sentence[fake_more_similar]
            return chat_bot_response
        
        else:           
            #if the search succeded returns the sentence with the highest similarity value into the info_sentences KB
            print("those are the info from trusted sources about the topic you asked: \n")

            chat_bot_response=info_sentence[info_more_similar]
            return chat_bot_response

In [ ]:
#interface
exit_codes=['bye', 'cya', 'exit', 'see you', 'thx', 'nope', 'no']
flag=True

print("welcome, I'm a program designed to help you verify information about sars2 covid-19.")
print("\n What would you like to do")

while(flag==True):
    print("\n add a confirmed hoax to the KB ->1")
    print("\n add a trusted information to the KB ->2")
    print("\n ask a question about covid-19 ->3")
    print("\n exit ->4")
    
    navigation=input()
    cycle="yes"
    
    if(navigation=="1"):
        user_response=input("your addition to the hoax KB>>")
        if user_response.lower() not in exit_codes:
            add_hoax(user_response.lower())
            print("\n\n would you like to do something else?\n")
            cycle=input()
           
    if(navigation=="2"):
        user_response=input("\n your addition to the info KB>>")
        if user_response.lower() not in exit_codes:
            add_info(user_response.lower())
            print("\n\n would you like to do something else? \n")
            cycle=input()
    
    if(navigation=="3"):
        user_response=input("your query>>")
        if user_response.lower() not in exit_codes:
            user_response=user_response.lower()
            print(response(user_response))
            info_sentence.remove(user_response)
            fake_sentence.remove(user_response)
            print("\n\n would you like to ask me something else?")
            cycle=input()
            
    if(navigation=="4"):
        flag=False
        
    if cycle.lower() in exit_codes:
        print("\n I hope I've been helpful, stay safe!")
        flag=False
        
    else:
        print("\n comand not found, try again")
        
#fakenews
    #Does taking a hot bath protect you?
    #Adding pepper at food cure Covid
#infotext
    #What are the symptoms?
    #How is the virus spread?

welcome, I'm a program designed to help you verify information about sars2 covid-19.

 What would you like to do

 add a confirmed hoax to the KB ->1

 add a trusted information to the KB ->2

 ask a question about covid-19 ->3

 exit ->4
Drinking alcohol does protect you against COVID-19

 comand not found, try again

 add a confirmed hoax to the KB ->1

 add a trusted information to the KB ->2

 ask a question about covid-19 ->3

 exit ->4
3
your query>>Drinking alcohol does protect you against COVID-19
those are the info from trusted sources about the topic you asked: 

[251] School closures, lockdowns, and reduced access to healthcare following the COVID-19 pandemic may differentially affect the genders and possibly exaggerate existing gender disparity.


 would you like to ask me something else?
3

 comand not found, try again

 add a confirmed hoax to the KB ->1

 add a trusted information to the KB ->2

 ask a question about covid-19 ->3

 exit ->4
3
your query>>Can eating garli

In [1]:
#print(info_sentence)
#print(fake_sentence)